In [1]:
import sys
import os

# Get the parent directory of the current folder
project_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_dir not in sys.path:
    sys.path.insert(0, project_dir)


In [2]:
import numpy as np
import plotly.graph_objects as go
from scipy.optimize import minimize
from tqdm import tqdm
from scipy.stats import t, multivariate_normal, multivariate_t
from scipy.special import logsumexp, gamma, gammaln
from numpy.linalg import inv, det, slogdet, LinAlgError
import plotly.express as px
from plotly.subplots import make_subplots
from numpy import tril_indices, triu_indices
from typing import Literal, Optional, Dict, Union

# from simulation import * 

from Functions.simulation_1 import * 
# from Functions.gas_filter_2 import *
# from Functions.kf_filter_2 import *
from Functions.gas_filter_5 import *
from Functions.kf_filter_5 import *

## $\Phi$ Full, $Q, R$ Diagonal

In [ ]:
# Explicit parameters with regressors
T = 500
N = 3
K = 3

c_true_3x3 = np.array([0.2, -0.1, 0.4])

# Phi_true_3x3_phi_full_no_noise_correl = np.array([[0.8, 0.2, 0],
#                                                     [0.1, 0.6, 0.2],
#                                                     [0, 0.2, 0.6]])
Phi_true_3x3_phi_full_no_noise_correl = np.array([[0.8, 0.2, 0.1],
                                                    [0.1, 0.5, 0.1],
                                                    [0.1, 0.2, 0.6]])

beta_true_3x3 = np.array([[1.0, -0.5, 0.2],
                          [-1.2, 0.8, 0.4],
                          [0.5, 1.5, -0.3]])

Q_true_3x3_no_correl = np.array([[0.6, 0.0, 0.0],
                                 [0.0, 0.5, 0.0],
                                 [0.0, 0.0, 0.6]])

R_true_3x3_no_correl = np.array([[0.4, 0.0, 0.0],
                                 [0.0, 0.7, 0.0],
                                 [0.0, 0.0, 0.5]])


y_sim_3x3_phi_full_no_noise_correl, mu_sim_3x3_phi_full_no_noise_correl, X_sim_3x3_phi_full_no_noise_correl = simulate_multivariate_state_space(
    T=T,
    N=N,
    c=c_true_3x3,
    Phi=Phi_true_3x3_phi_full_no_noise_correl,
    beta=beta_true_3x3, 
    Q=Q_true_3x3_no_correl,
    R=R_true_3x3_no_correl,
    seed=8888
)

In [4]:
gas_results_3x3_phi_full_no_correl_1 = estimate_and_filter_gas(
    y=y_sim_3x3_phi_full_no_noise_correl, 
    X=X_sim_3x3_phi_full_no_noise_correl,
    phi_type= "full",
    kappa_type= "full",
    fix_nu=None,
    burn_in=0.1,
    standardize=False
    )


mu_gas_3x3_phi_full_no_correl_1 = gas_results_3x3_phi_full_no_correl_1["mu_filtered"]
u_gas_3x3_phi_full_no_correl_1 = gas_results_3x3_phi_full_no_correl_1["u"]
resid_gas_phi_full_no_correl_1 = gas_results_3x3_phi_full_no_correl_1["resid"]
Kappa_est_3x3_phi_full_no_correl_1 = gas_results_3x3_phi_full_no_correl_1["Kappa"]

Estimated omega:
 [-6.8872506 -0.2360242 -4.8089412]
Estimated Phi:
 [[0.77802605 0.25540028 0.15140002]
 [0.12232875 0.31739197 0.10671188]
 [0.04258528 0.4168944  0.55287937]]
Estimated beta:
 [[10.22618204  0.43699356  6.85617807]
 [-1.16764507  0.75396422  0.41576916]
 [ 0.55773181  1.45925196 -0.30659643]]
Estimated Omega (from lambda):
 [[1.08055223 0.         0.        ]
 [0.         1.31375227 0.        ]
 [0.         0.         1.28295582]]
Estimated Kappa:
 [[0.59551316 0.08971509 0.01056899]
 [0.05414416 0.34870862 0.08662614]
 [0.04635298 0.13845598 0.44618735]]
Estimated nu:
 209.17195288460545
Log-likelihood: -2284.9374701695315
AIC: 4637.874940339063
BIC: 4818.5244335001335
Post-burn-in (after 50 obs):
Cut log-likelihood: -2067.947626921944
Cut AIC: 4203.895253843888
Cut BIC: 4380.962489472592


In [5]:
-2349.006250255705

-2357.818275401731

-2357.818275401731

In [6]:
# Dimensions
N = y_sim_3x3_phi_full_no_noise_correl.shape[1]
K = X_sim_3x3_phi_full_no_noise_correl.shape[1]

# Custom starting values
omega_init = np.array([0.1, 0.0, 0.2])                 # Shape (N,)
Phi_init = 0.5 * np.eye(N).flatten()                   # Shape (N*N,)
beta_init = 0.5 * np.ones((K, N)).flatten()                 # Shape (K*N,)

# Cholesky lower-triangle of Q and R (unconstrained)
L_Q_init = np.ones(N * (N + 1) // 2)                   # Length N*(N+1)/2
L_R_init = np.ones(N * (N + 1) // 2)

# Concatenate into full initial parameter vector
initial_params = np.concatenate([omega_init, Phi_init, beta_init, L_Q_init, L_R_init])

kf_results_3x3_phi_full_no_correl  =  multivariate_KF_with_estimation(
    y=y_sim_3x3_phi_full_no_noise_correl,
    X=X_sim_3x3_phi_full_no_noise_correl,
    initial_params=initial_params,
    verbose=True,
    burn_in=0.1,
    standardize=False,
)

mu_kf_3x3_phi_full_no_correl = kf_results_3x3_phi_full_no_correl['mu_filtered']
P_kf_3x3_phi_full_no_correl = kf_results_3x3_phi_full_no_correl['P_filtered']
kalman_gain_kf_3x3_phi_full_no_correl = kf_results_3x3_phi_full_no_correl['kalman_gain']

✅ Kalman estimation completed successfully.
Estimated parameters:
omega (unconditional mean): [1.01494964 0.16425781 0.73854214]
Phi (persistence matrix): 
[[0.82218564 0.14561248 0.10103772]
 [0.09095965 0.44068158 0.08991026]
 [0.06824538 0.22142082 0.60868747]]
beta: 
[[ 2.21863510e+00  1.71213108e-03  1.26741993e+00]
 [-1.17229002e+00  7.53341474e-01  4.09022506e-01]
 [ 5.56635866e-01  1.46096738e+00 -2.97684175e-01]]
Q (state noise covariance): 
[[0.54007548 0.0007349  0.0007349 ]
 [0.0007349  0.66564234 0.00081687]
 [0.0007349  0.00081687 0.58740561]]
R (observation noise covariance): 
[[0.37674154 0.00061379 0.04535651]
 [0.00061379 0.59243522 0.00084359]
 [0.04535651 0.00084359 0.57459848]]
Log-likelihood: -2295.5003284677837
AIC: 4657.000656935567
BIC: 4832.336929709547
Post-burn-in (after 50 obs):
Cut log-likelihood: -2078.2795630774617
Cut AIC: 4222.559126154923
Cut BIC: 4394.418501912195


In [ ]:
import pandas as pd

In [ ]:
gas_specs = [
    {"phi_type": "diagonal", "kappa_type": "diagonal", "fix_nu": None, "label": "GAS_diag_diag_nufree"},
    {"phi_type": "diagonal", "kappa_type": "diagonal", "fix_nu": 5, "label": "GAS_diag_diag_nu10"},
    {"phi_type": "diagonal", "kappa_type": "diagonal", "fix_nu": 10, "label": "GAS_diag_diag_nu10"},
    {"phi_type": "diagonal", "kappa_type": "diagonal", "fix_nu": 20, "label": "GAS_diag_diag_nu20"},
    {"phi_type": "diagonal", "kappa_type": "diagonal", "fix_nu": 50, "label": "GAS_diag_diag_nu50"},
    {"phi_type": "diagonal", "kappa_type": "diagonal", "fix_nu": 100, "label": "GAS_diag_diag_nu100"},
    {"phi_type": "diagonal", "kappa_type": "diagonal", "fix_nu": 500, "label": "GAS_diag_diag_nu500"},
    {"phi_type": "diagonal", "kappa_type": "full", "fix_nu": None, "label": "GAS_diag_full_nufree"},
    {"phi_type": "diagonal", "kappa_type": "full", "fix_nu": 5, "label": "GAS_diag_full_nu5"},
    {"phi_type": "diagonal", "kappa_type": "full", "fix_nu": 10, "label": "GAS_diag_full_nu10"},
    {"phi_type": "diagonal", "kappa_type": "full", "fix_nu": 20, "label": "GAS_diag_full_nu20"},
    {"phi_type": "diagonal", "kappa_type": "full", "fix_nu": 50, "label": "GAS_diag_full_nu50"},
    {"phi_type": "diagonal", "kappa_type": "full", "fix_nu": 100, "label": "GAS_diag_full_nu100"},
    {"phi_type": "diagonal", "kappa_type": "full", "fix_nu": 500, "label": "GAS_diag_full_nu500"},
    {"phi_type": "full", "kappa_type": "diagonal", "fix_nu": None, "label": "GAS_full_diag_nufree"},
    {"phi_type": "full", "kappa_type": "diagonal", "fix_nu": 5, "label": "GAS_full_diag_nu5"},
    {"phi_type": "full", "kappa_type": "diagonal", "fix_nu": 10, "label": "GAS_full_diag_nu10"},
    {"phi_type": "full", "kappa_type": "diagonal", "fix_nu": 20, "label": "GAS_full_diag_nu20"},
    {"phi_type": "full", "kappa_type": "diagonal", "fix_nu": 50, "label": "GAS_full_diag_nu50"},
    {"phi_type": "full", "kappa_type": "diagonal", "fix_nu": 100, "label": "GAS_full_diag_nu100"},
    {"phi_type": "full", "kappa_type": "diagonal", "fix_nu": 500, "label": "GAS_full_diag_nu500"},
    {"phi_type": "full", "kappa_type": "full", "fix_nu": None, "label": "GAS_full_full_nufree"},
    {"phi_type": "full", "kappa_type": "full", "fix_nu": 5, "label": "GAS_full_full_nu5"},
    {"phi_type": "full", "kappa_type": "full", "fix_nu": 10, "label": "GAS_full_full_nu10"},
    {"phi_type": "full", "kappa_type": "full", "fix_nu": 20, "label": "GAS_full_full_nu20"},
    {"phi_type": "full", "kappa_type": "full", "fix_nu": 50, "label": "GAS_full_full_nu50"},
    {"phi_type": "full", "kappa_type": "full", "fix_nu": 100, "label": "GAS_full_full_nu100"},
    {"phi_type": "full", "kappa_type": "full", "fix_nu": 500, "label": "GAS_full_full_nu500"},
]

In [ ]:
from collections import defaultdict

spec_results = defaultdict(list)
target_success = 30
sim_counter = 0

while any(len(v) < target_success for v in spec_results.values()) or len(spec_results) == 0:
    sim_seed = 8888 + sim_counter
    sim_counter += 1
    # Simulate data
    y_sim, mu_sim, X_sim = simulate_multivariate_state_space(
            T=1000,
            N=N,
            c=c_true_3x3,
            Phi=Phi_true_3x3_phi_full_no_noise_correl,
            beta=beta_true_3x3,  
            Q=Q_true_3x3_no_correl,
            R=R_true_3x3_no_correl,
            seed=sim_seed,
        )
    # KF
    try:
        if len(spec_results["KF"]) < target_success:
            kf_results = multivariate_KF_with_estimation(
            y=y_sim, 
            X=X_sim, 
            initial_params=None, 
            verbose=False,
            ftol=1e-8,  # Tolerance for convergence
            gtol=1e-6,  # Gradient tolerance for convergence,
        )
            # Calculate MSE for mu
            mu_kf = kf_results['mu_filtered']
            mse_kf = np.mean((mu_sim[1:] - mu_kf[1:]) ** 2)
            mae_kf = np.mean(np.abs(mu_sim[1:] - mu_kf[1:]))

            spec_results["KF"].append({
                "seed": sim_seed, "loglik": kf_results["loglik"],
                "aic": kf_results["aic"], "bic": kf_results["bic"],
                "mse": mse_kf, "mae": mae_kf
                })

    except Exception as e:
        print(f"KF failed: {e}")
    # GAS
    for spec in gas_specs:
        try:
            label = spec["label"]
            if len(spec_results[label]) < target_success:
                res_gas = estimate_and_filter_gas(
                    y=y_sim, 
                    X=X_sim,
                    phi_type=spec["phi_type"], 
                    kappa_type=spec["kappa_type"],
                    fix_nu=spec["fix_nu"], 
                    verbose=False,
                    ftol=1e-8,  # Tolerance for convergence
                    gtol=1e-6   # Gradient tolerance for convergence
                )

                # Calculate MSE for mu
                mu_gas = res_gas["mu_filtered"]
                mse_gas = np.mean((mu_sim[1:] - mu_gas[1:]) ** 2)
                mae_gas = np.mean(np.abs(mu_sim[1:] - mu_gas[1:]))

                spec_results[label].append({
                    "seed": sim_seed, "loglik": res_gas["loglik"],
                    "aic": res_gas["aic"], "bic": res_gas["bic"],
                    "mse": mse_gas, "mae": mae_gas
                })
        except Exception as e:
            print(f"GAS {label} failed: {e}")

    # Break if all filled
    if all(len(v) >= target_success for v in spec_results.values()):
        break


In [ ]:
import pandas as pd


rows = []
for spec_label, run_list in spec_results.items():
    for run in run_list:
        row = {'spec': spec_label}
        row.update(run) 
        rows.append(row)

df_results = pd.DataFrame(rows)


agg = df_results.groupby('spec').agg({
    'loglik': ['mean', 'std'],
    'aic': ['mean', 'std'],
    'bic': ['mean', 'std'],
    'mse': ['mean', 'std'],
    'mae': ['mean', 'std'],  
}).reset_index()
agg.columns = ['spec', 'loglik_mean', 'loglik_std', 'aic_mean', 'aic_std', 'bic_mean', 'bic_std', 'mse_mean', 'mse_std', 'mae_mean', 'mae_std']  # Added mse and mae


agg

,spec,loglik_mean,loglik_std,aic_mean,aic_std,bic_mean,bic_std,mse_mean,mse_std,mae_mean,mae_std
0,GAS_diag_diag_nu10,-4696.142067,49.158025,9434.284134,98.316050,9560.417853,98.316050,3.931063,0.750902,1.752545,0.174192
1,GAS_diag_diag_nu100,-4624.148567,31.763784,9290.297134,63.527568,9416.430853,63.527568,1.989806,0.327384,1.243904,0.110601
2,GAS_diag_diag_nu20,-4635.498938,32.482361,9312.997877,64.964721,9439.131596,64.964721,3.074909,0.559421,1.555708,0.147837
3,GAS_diag_diag_nu50,-4625.884691,31.942172,9293.769381,63.884343,9419.903100,63.884343,2.286328,0.397882,1.337709,0.123648
4,GAS_diag_diag_nu500,-4623.514525,31.633323,9289.029050,63.266646,9415.162769,63.266646,1.756838,0.258787,1.164381,0.094289
5,GAS_diag_diag_nufree,-4623.607419,31.934440,9291.214838,63.868881,9423.354924,63.868881,56.450095,28.429605,6.049451,2.008827
6,GAS_diag_full_nu10,-4606.122032,32.497001,9266.244064,64.994001,9428.415988,64.994001,3.725985,0.628666,1.704968,0.149622
7,GAS_diag_full_nu100,-4573.480935,32.209154,9200.961870,64.418308,9363.133794,64.418308,1.997996,0.288357,1.238892,0.098116
8,GAS_diag_full_nu20,-4583.520308,32.309426,9221.040616,64.618853,9383.212540,64.618853,3.125664,0.566665,1.561813,0.148237
9,GAS_diag_full_nu5,-4663.521770,33.084796,9381.043539,66.169592,9543.215464,66.169592,4.323570,0.724362,1.830721,0.159940


In [16]:
print(round(agg, 2).to_latex())

\begin{tabular}{llrrrrrrrrrr}
\toprule
 & spec & loglik_mean & loglik_std & aic_mean & aic_std & bic_mean & bic_std & mse_mean & mse_std & mae_mean & mae_std \\
\midrule
0 & GAS_diag_diag_nu10 & -4696.140000 & 49.160000 & 9434.280000 & 98.320000 & 9560.420000 & 98.320000 & 3.930000 & 0.750000 & 1.750000 & 0.170000 \\
1 & GAS_diag_diag_nu100 & -4624.150000 & 31.760000 & 9290.300000 & 63.530000 & 9416.430000 & 63.530000 & 1.990000 & 0.330000 & 1.240000 & 0.110000 \\
2 & GAS_diag_diag_nu20 & -4635.500000 & 32.480000 & 9313.000000 & 64.960000 & 9439.130000 & 64.960000 & 3.070000 & 0.560000 & 1.560000 & 0.150000 \\
3 & GAS_diag_diag_nu50 & -4625.880000 & 31.940000 & 9293.770000 & 63.880000 & 9419.900000 & 63.880000 & 2.290000 & 0.400000 & 1.340000 & 0.120000 \\
4 & GAS_diag_diag_nu500 & -4623.510000 & 31.630000 & 9289.030000 & 63.270000 & 9415.160000 & 63.270000 & 1.760000 & 0.260000 & 1.160000 & 0.090000 \\
5 & GAS_diag_diag_nufree & -4623.610000 & 31.930000 & 9291.210000 & 63.870000 & 942